In [5]:
from azure.ml import MLClient
from azure.identity import DefaultAzureCredential

In [6]:
# subscription_id = "6560575d-fa06-4e7d-95fb-f962e74efd7a"
# resource_group = "azureml-examples"
# workspace = "main"

subscription_id = "15ae9cb6-95c1-483d-a0e3-b1a1a3b06324"
resource_group = "ray"
workspace = "ray"



ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

## Command Job

In [7]:
from azure.ml import command, Input, Output, PyTorchDistribution
from azure.ml.entities import ResourceConfiguration, Environment 
prep = command(
  code='src',
  command=
    "python startDask.py "
    "--script prep-nyctaxi.py "
    "--nyc_taxi_dataset ${{inputs.nyc_taxi_dataset}} "
    "--output_folder ${{outputs.output_folder}}",
  inputs={
    'nyc_taxi_dataset': Input(
        path= 'wasbs://datasets@azuremlexamples.blob.core.windows.net/nyctaxi/',
        mode= 'ro_mount')},
  outputs={
    'output_folder':Output(
      type= 'uri_folder')},
  environment=Environment( 
    image= 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04',
    conda_file= 'conda.yml'),
  compute= 'daniel-big',
  resources=ResourceConfiguration(instance_count=4),
  distribution=PyTorchDistribution(),
  experiment_name= 'dask-nyctaxi-pipeline-example',
  description= 'This sample shows how to run a distributed DASK job on AzureML. The 24GB NYC Taxi dataset is read in CSV format by a 4 node DASK cluster, processed and then written as job output in parquet format.'
)

In [8]:
train = command(
  code= 'src',
  command=
    "python train-xgboost.py "
    "--nyc_taxi_parquet ${{inputs.nyc_taxi_parquet}} "
    "--model ${{outputs.model}} "
    "--tree_method ${{inputs.tree_method}} "
    "--learning_rate ${{inputs.learning_rate}} "
    "--gamma ${{inputs.gamma}} "
    "--max_depth ${{inputs.max_depth}} "
    "--num_boost_round ${{inputs.num_boost_round}} ",
  inputs={
    "nyc_taxi_parquet": Input(
      path='azureml:azureml_polite_loquat_c3x4fj4l4m_output_data_output_folder:1',
      mode='ro_mount'),
    "tree_method": "auto",
    "learning_rate": 0.3,
    "gamma": 1,
    "max_depth": 7,
    "num_boost_round": 20,
  },
  outputs={
    "model": Output(type='mlflow_model')
  },
  environment=Environment(
    image= "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file= "conda.yml"),
  compute= "daniel-big",
  experiment_name= "dask-nyctaxi-example"
)

In [9]:
from azure.ml import command, Input, Output
from azure.ml.entities import Environment

test = command(
  code="src",
  command=
    "cp -r ${{inputs.model_in}}/* ${{outputs.model_out}} && "
    "echo testing MLFLow model && "
    "mlflow models predict -m ${{outputs.model_out}} -i ${{inputs.testdata}} -t json && "
    "echo && "
    "cat ${{outputs.model_out}}/MLmodel",
  inputs={
    "model_in": Input(
      path='data/fare_predict'),
    "testdata": Input(
      path='data/data.json',
      type='uri_file')
  },
  outputs={
    "model_out": Output(type='mlflow_model')
  },
  environment=Environment(
    image= "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file= "conda.yml"),
  compute= "cpu-cluster",
  experiment_name= "dask-nyctaxi-example"
)

In [10]:
ml_client.jobs.create_or_update(test)

Uploading src (0.02 MBs): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17559/17559 [00:00<00:00, 511627.70it/s]


Experiment,Name,Type,Status,Details Page
dask-nyctaxi-example,stoic_house_drw1xxlw0x,command,Starting,Link to Azure Machine Learning studio


## Pipeline Job

In [11]:
from azure.ml.dsl import pipeline

@pipeline()
def prep_train_test(dataset: Input):
  prep_job = prep(nyc_taxi_dataset=dataset)
  train_job = train(nyc_taxi_parquet=prep_job.outputs.output_folder,
                    tree_method='auto',
                    learning_rate= 0.3,
                    gamma= 1,
                    max_depth= 7,
                    num_boost_round= 12)
  test_job = test(model_in=train_job.outputs.model)
  return dict(model=test_job.outputs.model_out)

In [12]:
nyc_raw_data = Input(path='wasbs://datasets@azuremlexamples.blob.core.windows.net/nyctaxi/')

pipeline_job = prep_train_test(dataset=nyc_raw_data)

ml_client.jobs.create_or_update(pipeline_job)

Experiment,Name,Type,Status,Details Page
e2e-dask-sweep,funny_bucket_xjscw9zj7w,pipeline,Preparing,Link to Azure Machine Learning studio


## Sweep Job

In [12]:
from azure.ml.dsl import pipeline
from azure.ml.sweep import Choice, Uniform

@pipeline()
def prep_sweep_test(dataset: Input):
  prep_job = prep(nyc_taxi_dataset=dataset)

  train_job = train(nyc_taxi_parquet=prep_job.outputs.output_folder,
                    tree_method=Choice(['auto', 'exact', 'approx', 'hist']),
                    learning_rate=Uniform(0, 1),
                    gamma= Choice(range(7)),
                    max_depth= Choice(range(4,8)),
                    num_boost_round= 20,)

  sweep_job = train_job.sweep(primary_metric='test-rmse',
                              goal='minimize',
                              sampling_algorithm='bayesian',
                              compute='daniel-big')

  sweep_job.set_limits(max_concurrent_trials=5,
                      max_total_trials=25)

  test_job = test(model_in=sweep_job.outputs.model)
  return dict(model=test_job.outputs.model_out)

In [13]:
nyc_raw_data = Input(path= 'wasbs://datasets@azuremlexamples.blob.core.windows.net/nyctaxi/')

pipeline_job = prep_sweep_test(dataset=nyc_raw_data)

ml_client.jobs.create_or_update(pipeline_job)

Experiment,Name,Type,Status,Details Page
e2e-dask-sweep,crimson_guava_4drxl2zm27,pipeline,Preparing,Link to Azure Machine Learning studio


## Standalone Sweep Job

In [21]:
from azure.ml.sweep import Choice, Uniform, LogUniform

train_job = train(tree_method=Choice(['auto', 'exact', 'approx', 'hist']),
                  learning_rate=Uniform(0, 1),
                  gamma= Choice(range(7)),
                  max_depth= Choice(range(4,8)),
                  num_boost_round= 20,)

In [22]:
from azure.ml.sweep import BayesianSamplingAlgorithm

sweep_job = train_job.sweep(primary_metric='test-rmse',
                            goal='minimize',
                            sampling_algorithm=BayesianSamplingAlgorithm(),
                            compute='daniel-big')

sweep_job.set_limits(max_concurrent_trials=5,
                     max_total_trials=25)


In [23]:
ml_client.jobs.create_or_update(sweep_job)

Experiment,Name,Type,Status,Details Page
e2e-dask-sweep,9290a448-867f-cc19-df3b-90dfeb28619a,sweep,Running,Link to Azure Machine Learning studio
